# <font color = 'red'> Pré Processamento e Engenharia de Atributos</font>

### <font color = 'red'> Foco: </font>

- Pré processar o texto para a etapa de análise de sentimentos

- Remoção de stopwords e outros, para verificar ocorrência

In [1]:
# Imports
import os
import re
import numpy as np
import pandas as pd

In [13]:
# Visualizando Cj dados
dados = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/data_scraping_bruto.csv',
                       sep =',')

In [14]:
dados.head()

,Empresa,Relato,Avaliacao
0,Serasa Experian,C. A legislação consumerista (à luz da qual de...,Nota 5
1,Serasa Experian,Solicito a baixa dos meus dados junto ao SERAS...,Nota 1
2,Serasa Experian,Eu solicito a limpeza em seus cadastros,Nota 5
3,Latam Airlines (Tam),"Prezados, apesar de ser cliente da categoria g...",Nota 5
4,Serasa Experian,Em 04/11/2024 verifiquei o extrato da conta co...,Nota 1


In [15]:
instituicoes_financeiras = ['Nubank',
              'Banco do Brasil',
              'Caixa Econômica Federal',
              'Banco Santander',
              'Banco Itaú Unibanco',
              'Mercado Pago',
              'Banco Inter (Banco Intermedium)',
              'Banco Bradesco',
              'C6 Bank']

In [16]:
# Selecionando apenas instituições financeiras

dados_financeiros = dados[dados['Empresa'].isin(instituicoes_financeiras)]

In [17]:
#Verificando ocorrências

dados_financeiros.Empresa.value_counts().reset_index(name='Quantidade Relatos')

,Empresa,Quantidade Relatos
0,Nubank,484
1,Banco do Brasil,365
2,Caixa Econômica Federal,242
3,Banco Santander,212
4,Banco Itaú Unibanco,142
5,Mercado Pago,139
6,Banco Bradesco,137
7,Banco Inter (Banco Intermedium),127
8,C6 Bank,72


In [18]:
dados_financeiros.head(10)

,Empresa,Relato,Avaliacao
23,Caixa Econômica Federal,Não consigo crédito pra fazer um empréstimo me...,Nota 3
41,Nubank,ja houve o pagamento da negociacao,Nota 2
45,Nubank,Exclusao do nome Retirar negativacao do nome s...,Nota 1
49,C6 Bank,"Sou cliente do C6 Bank há tempos, sempre mante...",Nota 1
52,Nubank,Gostaria de reclamar sobre essa dívida quer Es...,Nota 5
58,Banco Itaú Unibanco,O Itaú financiamentos fez Constar na SCR Uma l...,Nota 1
59,Nubank,Eu fiz um acordo para pagamento da minha dívid...,Nota 3
69,Banco Itaú Unibanco,"Primeiramente quero agradecer o banco itau, po...",Nota 5
73,Nubank,Reclamação Nubank Gostaria de registrar minha ...,Nota 1
81,Caixa Econômica Federal,Desejo participar do mutirão de renegociação.,Nota 1


In [24]:
dados_financeiros['Avaliacao'] = dados_financeiros['Avaliacao'].str.extract(r'(\d+)')

C:\Users\alyss\AppData\Local\Temp\ipykernel_20312\3650314984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_financeiros['Avaliacao'] = dados_financeiros['Avaliacao'].str.extract(r'(\d+)')


In [26]:
dados_financeiros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1920 entries, 23 to 10929
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Empresa    1920 non-null   object
 1   Relato     1920 non-null   object
 2   Avaliacao  1920 non-null   object
dtypes: object(3)
memory usage: 60.0+ KB


### <font color='green'> Remoção de pontuações e deixando o texto em caixa baixa

In [27]:
def processa_sentimento(text):
 # Minuscula
 text = text.lower()

 # retirando pontuações, exceto "."
 text = re.sub(r'[^\w\s.]', '', text)

 return text

In [28]:
dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(processa_sentimento)

C:\Users\alyss\AppData\Local\Temp\ipykernel_20312\750078115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(processa_sentimento)


In [29]:
dados_financeiros.sample(5)

,Empresa,Relato,Avaliacao
4944,Banco do Brasil,solicito exclusão dos meus dados do relatório ...,1
9774,Mercado Pago,negativação indevida pois não há divida na ins...,5
10809,Banco Bradesco,no dia 17 de outubro de 2024 fui vítima de um ...,1
6355,Banco Bradesco,aplicação da resolução 4.7902020 capítulo iv. ...,5
10478,Banco Bradesco,solicito um documento oficial do bradesco que ...,5


In [30]:
dados_financeiros.to_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',', index = False, encoding = 'utf-8')


### <font color='green'> Pré processamento para frequência